In [150]:
import pandas as pd # type: ignore
from datetime import datetime # type: ignore

In [151]:
import glob

folder_path = 'datasets/LJP'

# Get a list of all CSV files in the folder
csv_files = glob.glob(folder_path + '/*.csv')

# Create an empty list to store the DataFrames
dfs = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
	df = pd.read_csv(file)
	dfs.append(df)

# Concatenate all the DataFrames together
df_arso = pd.concat(dfs, ignore_index=True)
df_arso.head()

,station id,station name,valid,T [°C],količina padavin [mm],globalno sev. [W/m2]
0,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:00,-2.6,0.0,0.0
1,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:10,-2.6,NaN,NaN
2,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:20,-2.6,NaN,NaN
3,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:30,-3.4,0.0,0.0
4,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:40,-2.9,NaN,NaN


In [152]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_arso):
	# Filter rows based on column: ' valid'
	df_arso = df_arso[df_arso[' valid'].str.endswith("00", na=False)]
	# Drop columns: 'station id', ' station name'
	df_arso = df_arso.drop(columns=['station id', ' station name'])
	# Change column type to datetime64[ns] for column: ' valid'
	df_arso = df_arso.astype({' valid': 'datetime64[ns]'})
	# Rename column ' valid' to 'datum'
	df_arso = df_arso.rename(columns={' valid': 'datum'})
	return df_arso

df_arso_clean = clean_data(df_arso.copy())
df_arso_clean.head()

,datum,T [°C],količina padavin [mm],globalno sev. [W/m2]
0,2022-01-01 00:00:00,-2.6,0.0,0.0
6,2022-01-01 01:00:00,-3.2,0.0,0.0
12,2022-01-01 02:00:00,-3.9,0.0,0.0
18,2022-01-01 03:00:00,-2.8,0.0,0.0
24,2022-01-01 04:00:00,-3.1,0.0,0.0


In [153]:
df_eles = pd.read_csv('datasets/main/ELES.csv', sep=';')

"""
Cell generated by Data Wrangler.
"""
from datetime import datetime

def clean_data(df_eles):
    df_eles['datum'] = pd.to_datetime(df_eles['datum'], dayfirst=True)
    df_eles['ura'] = df_eles['ura'].str[1:3].astype(int) - 1
    df_eles['datum'] = df_eles['datum'] + pd.to_timedelta(df_eles['ura'], unit='h')
    df_eles.drop(columns=['ura', 'predvidena_proizvodnja', 'dejanska_proizvodnja', 'nek', 'termo', 'hidro', 'predviden_prevzem'], inplace=True)
    # Filter rows based on column: 'datum'
    # df_eles = df_eles[df_eles['datum'] >= datetime.strptime('2024-08-01T00:00:00.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')]
    return df_eles

df_eles_clean = clean_data(df_eles.copy())
df_eles_clean.head()

,datum,dejanski_prevzem
0,2022-01-01 00:00:00,1044.0
1,2022-01-01 01:00:00,1018.0
2,2022-01-01 02:00:00,975.0
3,2022-01-01 03:00:00,944.0
4,2022-01-01 04:00:00,936.0


In [154]:
df_pokritost = pd.read_csv('datasets/main/LJP_pokritost_2022-2024.csv')
df_pokritost['datum'] = pd.to_datetime(df_pokritost[' valid'])
df_pokritost = df_pokritost.drop(columns=['station id', ' station name', ' valid'])
df_pokritost.head()

,oblačnost [%],datum
0,37.0,2022-01-01
1,40.0,2022-01-02
2,80.0,2022-01-03
3,100.0,2022-01-04
4,100.0,2022-01-05


In [155]:
from prophet.make_holidays import make_holidays_df # type: ignore

df_prazniki = make_holidays_df(
    year_list=[2022 + i for i in range(3)], country='SI'
)
df_prazniki = df_prazniki.rename(columns={'ds': 'datum'})
df_prazniki.sort_values('datum', inplace=True)
df_prazniki['datum'] = pd.to_datetime(df_prazniki['datum'])

# Generate a new DataFrame with all dates between 2022-01-01 and 2024-10-23
df_all_dates = pd.DataFrame({'datum': pd.date_range('2022-01-01', '2024-12-31')})

# Merge the new DataFrame with df_prazniki, filling in missing values with 0
df_prazniki = pd.merge(df_all_dates, df_prazniki, on='datum', how='outer').fillna(0)

df_prazniki.head()

,datum,holiday
0,2022-01-01,New Year's Day
1,2022-01-02,New Year's Day
2,2022-01-03,0
3,2022-01-04,0
4,2022-01-05,0


In [156]:
df_prices = pd.read_csv('datasets/main/SIPX_hourly_data.csv')
df_prices['DeliveryDateTime'] = pd.to_datetime(df_prices['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
df_prices['DeliveryDateTime'] = df_prices['DeliveryDateTime'].astype('datetime64[ns]')
df_prices = df_prices.rename(columns={'DeliveryDateTime': 'datum', 'Price': 'price'})
df_prices.tail()

C:\Users\patri\AppData\Local\Temp\ipykernel_23096\2107259201.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_prices['DeliveryDateTime'] = pd.to_datetime(df_prices['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))


,datum,price
24801,2024-10-30 19:00:00,155.82
24802,2024-10-30 20:00:00,142.13
24803,2024-10-30 21:00:00,126.32
24804,2024-10-30 22:00:00,116.97
24805,2024-10-30 23:00:00,113.16


In [157]:
df1 = pd.merge(df_eles_clean, df_arso_clean, on='datum', how='inner')
df2 = pd.merge(df1, df_prices, on='datum', how='left')
df2

,datum,dejanski_prevzem,T [°C],količina padavin [mm],globalno sev. [W/m2],price
0,2022-01-01 00:00:00,1044.0,-2.6,0.0,0.0,NaN
1,2022-01-01 01:00:00,1018.0,-3.2,0.0,0.0,51.44327
2,2022-01-01 02:00:00,975.0,-3.9,0.0,0.0,55.91396
3,2022-01-01 03:00:00,944.0,-2.8,0.0,0.0,46.94279
4,2022-01-01 04:00:00,936.0,-3.1,0.0,0.0,37.67210
...,...,...,...,...,...,...
25360,2024-10-23 19:00:00,1583.0,14.4,0.0,0.0,191.94000
25361,2024-10-23 20:00:00,1523.0,13.4,0.5,0.0,132.71000
25362,2024-10-23 21:00:00,1428.0,12.9,0.8,0.0,106.98000
25363,2024-10-23 22:00:00,1308.0,12.6,1.3,0.0,103.02000


In [158]:
df3 = pd.merge(df_pokritost, df_prazniki, on='datum', how='left')
df3

,oblačnost [%],datum,holiday
0,37.0,2022-01-01,New Year's Day
1,40.0,2022-01-02,New Year's Day
2,80.0,2022-01-03,0
3,100.0,2022-01-04,0
4,100.0,2022-01-05,0
...,...,...,...
1022,100.0,2024-10-19,0
1023,53.0,2024-10-20,0
1024,37.0,2024-10-21,0
1025,50.0,2024-10-22,0


In [159]:
df = pd.merge(df2, df3, on='datum', how='left')
df

,datum,dejanski_prevzem,T [°C],količina padavin [mm],globalno sev. [W/m2],price,oblačnost [%],holiday
0,2022-01-01 00:00:00,1044.0,-2.6,0.0,0.0,NaN,37.0,New Year's Day
1,2022-01-01 01:00:00,1018.0,-3.2,0.0,0.0,51.44327,NaN,NaN
2,2022-01-01 02:00:00,975.0,-3.9,0.0,0.0,55.91396,NaN,NaN
3,2022-01-01 03:00:00,944.0,-2.8,0.0,0.0,46.94279,NaN,NaN
4,2022-01-01 04:00:00,936.0,-3.1,0.0,0.0,37.67210,NaN,NaN
...,...,...,...,...,...,...,...,...
25360,2024-10-23 19:00:00,1583.0,14.4,0.0,0.0,191.94000,NaN,NaN
25361,2024-10-23 20:00:00,1523.0,13.4,0.5,0.0,132.71000,NaN,NaN
25362,2024-10-23 21:00:00,1428.0,12.9,0.8,0.0,106.98000,NaN,NaN
25363,2024-10-23 22:00:00,1308.0,12.6,1.3,0.0,103.02000,NaN,NaN


In [160]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Drop duplicate rows in column: 'datum'
    df = df.drop_duplicates(subset=['datum'])
    # Renames
    df = df.rename(columns={'datum': 'date', 
                            'dejanski_prevzem': 'consumption',
                            'T [°C]': 'temperature',
                            'količina padavin [mm]': 'precipitation',
                            'globalno sev. [W/m2]': 'irradiation',
                            'oblačnost [%]': 'cloudiness'
                        })
    df.ffill(inplace=True)
    df = df[(df['date'].dt.day != 29) | (df['date'].dt.month != 2)]
    df = df.drop(df.index[0])
    return df

df = clean_data(df.copy())
df

,date,consumption,temperature,precipitation,irradiation,price,cloudiness,holiday
1,2022-01-01 01:00:00,1018.0,-3.2,0.0,0.0,51.44327,37.0,New Year's Day
2,2022-01-01 02:00:00,975.0,-3.9,0.0,0.0,55.91396,37.0,New Year's Day
3,2022-01-01 03:00:00,944.0,-2.8,0.0,0.0,46.94279,37.0,New Year's Day
4,2022-01-01 04:00:00,936.0,-3.1,0.0,0.0,37.67210,37.0,New Year's Day
5,2022-01-01 05:00:00,952.0,-4.1,0.0,0.0,39.70000,37.0,New Year's Day
...,...,...,...,...,...,...,...,...
25360,2024-10-23 19:00:00,1583.0,14.4,0.0,0.0,191.94000,97.0,0
25361,2024-10-23 20:00:00,1523.0,13.4,0.5,0.0,132.71000,97.0,0
25362,2024-10-23 21:00:00,1428.0,12.9,0.8,0.0,106.98000,97.0,0
25363,2024-10-23 22:00:00,1308.0,12.6,1.3,0.0,103.02000,97.0,0
